In [2]:
#normal imports of python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [29]:
'''
let me read the file information.
This is for myself to get clarity, I'll update the info in the seaprate file 
see the file named data_info on my github
'''

infor = pd.read_csv('../DATA/lending_club_info.csv')
infor

,LoanStatNew,Description
0,loan_amnt,The listed amount of the loan applied for by t...
1,term,The number of payments on the loan. Values are...
2,int_rate,Interest Rate on the loan
3,installment,The monthly payment owed by the borrower if th...
4,grade,LC assigned loan grade
5,sub_grade,LC assigned loan subgrade
6,emp_title,The job title supplied by the Borrower when ap...
7,emp_length,Employment length in years. Possible values ar...
8,home_ownership,The home ownership status provided by the borr...
9,annual_inc,The self-reported annual income provided by th...


In [40]:
# let me import the whole data
df = pd.read_csv('../DATA/lending_club_loan_two.csv');

In [41]:
#its been difficult to get the info of the whole data set every time
def whole_info(short_name):
    print(infor[infor['LoanStatNew']== short_name]['Description'])
    
    
    
# preprocessing,cleaning the data    
def create_dummies(strg):
    if strg == 'Fully Paid':
        return 1
    else:
        return 0
df['loan_repaid'] = df['loan_status'].apply(create_dummies)
df['term'] = df['term'].apply(lambda x: x.split(' ')[1])
df['term'] = df['term'].astype(int)
feat_info('emp_title')
feat_info('emp_length')
df.drop('emp_title',axis=1,inplace = True)
odr= df['emp_length'].dropna().unique()
order1 = ['< 1 year','1 year','2 years', '3 years','4 years','5 years','6 years', '7 years','8 years', '9 years','10+ years'   ]
df.drop('emp_length',axis=1,inplace=True)
df.drop('title',axis=1,inplace=True)
feat_info('mort_acc')
dtmean = df.groupby('total_acc')['mort_acc'].mean()
fvalue = []
df[df['mort_acc'].isnull()==True]['total_acc'].apply(lambda x: fvalue.append(dtmean[x]))
def fillup(mort,total):
    if np.isnan(mort):
        return dtmean[total]
    else:
        return mort     
df['mort_acc'] = df.apply(lambda x: fillup(x['total_acc'], x['mort_acc']), axis=1)

feat_info('revol_util')
df.dropna(inplace= True)
df.drop('grade', axis=1,inplace=True)
dummygrade = pd.get_dummies(df['sub_grade'],drop_first = True)
df_new = pd.concat([df,dummygrade], axis=1)
df_new.drop('sub_grade', axis=1)
df.select_dtypes(object)['home_ownership'].value_counts()
df['home_ownership'] = df['home_ownership'].replace(['NONE','ANY'], 'OTHER')
dummy_home = pd.get_dummies(df['home_ownership'], drop_first = True)
df_new = pd.concat([dummy_home,df_new],axis=1)
df_new.drop(['OTHER', 'OWN' ,'RENT', 'home_ownership'],axis=1,inplace =True)
df.drop('loan_status',axis=1,inplace=True)
df.drop(['address'],axis=1,inplace= True)
df_new.drop('sub_grade',axis=1,inplace=True)
df_new['zip code'] = df_new['address'].apply(lambda x: x[-5:])
dummy_zip = pd.get_dummies(data=df_new['zip code'])
df = pd.concat([df_new,dummy_zip],axis=1)
df.drop('zip code', axis=1,inplace= True)
df.drop('issue_d', axis=1,inplace= True)
df['earliest_cr_line'] = df['earliest_cr_line'].apply(lambda x: x[-4:])
pd.to_datetime(df['earliest_cr_line'])
df['earliest_cr_line'] = df['earliest_cr_line'].apply(lambda x: int(x))
dummy_all = pd.get_dummies(df[['verification_status','purpose', 'initial_list_status', 'application_type']],drop_first=True)
df.drop([ 'purpose','initial_list_status','application_type'],axis=1,inplace=True)
df.drop('verification_status',axis=1,inplace=True)
df = pd.concat([df,dummy_all],axis=1)

In [ ]:
#tensorflow model for the predicting the risk factor with the customer
# splitting the data for train,valifation,test. 
#due to lower clockspeed I skipped validation and use test data for both
from sklearn.model_selection import train_test_split
X = df.drop('loan_repaid',axis=1).values
y= df['loan_repaid'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=96)

In [ ]:
#scaling and transforming the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

In [ ]:
#standard inputs which I generally use
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.constraints import max_norm
from sklearn.metrics import classification_report,confusion_matrix 

In [ ]:
#creating the model for classification
model = Sequential()
model.add(Dense(77,activation ='relu'))
model.add(Dropout(0.3))
model.add(Dense(35,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(22,activation= 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation ='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
#fitting the data in the model
model.fit(x= X_train,
         y= y_train,
         epochs=50,
         batch_size=280,
         validation_data = (X_test,y_test),
         verbose = 1
         )

In [ ]:
#evaluation
losses = model.history.history
loss = pd.DataFrame(losses)
pred = model.predict_classes(X_test)
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

# the results are found to be 80% accurate

Results